<a href="https://colab.research.google.com/github/venkatchintha/Project0/blob/master/Monk_to_COCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Kaggle/nexet/nexet/

/content/drive/My Drive/Kaggle/nexet/nexet


In [2]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

Cloning into 'Monk_Object_Detection'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 6902 (delta 82), reused 62 (delta 31), pack-reused 6740
Receiving objects: 100% (6902/6902), 205.31 MiB | 32.95 MiB/s, done.
Resolving deltas: 100% (2693/2693), done.
Checking out files: 100% (5499/5499), done.


In [4]:
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

     |████████████████████████████████| 540.1MB 25kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Created wheel for dicttoxml: filename=dicttoxml-1.7.4-cp36-none-any.whl size=17452 sha256=b65b8d5dd345a8ee46010456bfbc38fc71bde366491ec131943ab4e856e01821
  Stored in directory: /root/.cache/pip/wheels/62/4f/a3/afd4a68f5add45a668c14efa53b64d5cffb2be6bacf993c151
Successfully built dicttoxml


In [5]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd

In [25]:
root= "/content/drive/My Drive/Kaggle/nexet/nexet";
img_dir = "nexet_2017_1/";
anno_file = "train_labels.csv";

In [26]:
dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

In [27]:
if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;

In [28]:
output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

In [29]:
if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);

In [30]:
df = pd.read_csv(input_annotations_path);
columns = df.columns

In [31]:
delimiter = " ";

In [32]:
list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();




100%|██████████| 5/5 [00:00<00:00, 14037.16it/s]


In [ ]:
coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(float(tmp[j*5+0]));
        y1 = int(float(tmp[j*5+1]));
        x2 = int(float(tmp[j*5+2]));
        y2 = int(float(tmp[j*5+3]));
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();




  0%|          | 0/49281 [00:00<?, ?it/s]


  0%|          | 1/49281 [00:00<3:01:05,  4.54it/s]


  0%|          | 2/49281 [00:00<3:19:51,  4.11it/s]


  0%|          | 3/49281 [00:00<3:31:36,  3.88it/s]


  0%|          | 4/49281 [00:01<3:25:12,  4.00it/s]


  0%|          | 5/49281 [00:01<3:26:53,  3.97it/s]


  0%|          | 6/49281 [00:01<3:20:34,  4.09it/s]


  0%|          | 7/49281 [00:01<3:26:53,  3.97it/s]


  0%|          | 8/49281 [00:01<3:14:32,  4.22it/s]


  0%|          | 9/49281 [00:02<3:09:56,  4.32it/s]


  0%|          | 10/49281 [00:02<3:22:36,  4.05it/s]


  0%|          | 11/49281 [00:03<4:26:59,  3.08it/s]


  0%|          | 12/49281 [00:03<4:06:03,  3.34it/s]


  0%|          | 13/49281 [00:03<3:54:22,  3.50it/s]


  0%|          | 14/49281 [00:03<3:39:21,  3.74it/s]


  0%|          | 15/49281 [00:04<3:54:28,  3.50it/s]


  0%|          | 16/49281 [00:04<3:43:51,  3.67it/s]


  0%|          | 17/49281 [00:04<3:28:12,  3.94it/s]


  0%|          | 18/49281 [